In [3]:
import networkx as nx
import numpy as np
from pytket import Qubit
from pytket.circuit import CircBox, Circuit, Qubit
from pytket.pauli import Pauli, QubitPauliString
from pytket.utils import QubitPauliOperator, gen_term_sequence_circuit
from scipy.linalg import expm
import matplotlib.pyplot as plt
from pytket.extensions.qiskit import AerStateBackend

from hlsim.utils import get_pauli_exp_box_from_QubitPauliString

ModuleNotFoundError: No module named 'hlsim'

In [ ]:
def get_xxz_chain_hamiltonian(n_qubits: int, Delta_ZZ: float) -> QubitPauliOperator:
    sites = nx.path_graph(n_qubits)
    qpo_dict = {}
    for e in sites.edges:
        zz_term = QubitPauliString([Qubit(e[0]), Qubit(e[1])], [Pauli.Z, Pauli.Z])
        xx_term = QubitPauliString([Qubit(e[0]), Qubit(e[1])], [Pauli.X, Pauli.X])
        yy_term = QubitPauliString([Qubit(e[0]), Qubit(e[1])], [Pauli.Y, Pauli.Y])
        qpo_dict[zz_term] = Delta_ZZ
        qpo_dict[xx_term] = 1.0
        qpo_dict[yy_term] = 1.0

    return QubitPauliOperator(qpo_dict)


def get_transverse_field_ising_hamiltonian(
    n_qubits: int, J_ZZ: float
) -> QubitPauliOperator:
    sites = nx.path_graph(n_qubits)
    qpo_dict = {}
    for e in sites.edges:
        zz_term = QubitPauliString([Qubit(e[0]), Qubit(e[1])], [Pauli.Z, Pauli.Z])
        qpo_dict[zz_term] = -J_ZZ
    for node in sites.nodes:
        x_term = QubitPauliString([Qubit(node)], [Pauli.X])
        qpo_dict[x_term] = 1.0

    return QubitPauliOperator(qpo_dict)

In [4]:
def hamiltonian_time_evolution_numpy(
    hamiltonian: QubitPauliOperator, t_trotterization: float, n_qubits
):
    mat = hamiltonian.to_sparse_matrix(
        [Qubit(idx) for idx in range(n_qubits)]
    ).todense()
    U = expm(-1j * t_trotterization * mat)
    return U


def get_hamiltonian_simulation_circbox(
    hamiltonian: QubitPauliOperator,
    n_qubits: int,
    t_trotterization: float,
    n_trotter_steps: int,
):
    trotter_step_size = t_trotterization / n_trotter_steps

    # Bug in docstring of gen_term_sequence_circuit -> no minus required!
    scaled_hamiltonian = trotter_step_size * (2 / np.pi) * hamiltonian

    time_evo_circ = Circuit(n_qubits=n_qubits, name="Time evolution")
    base_circ = Circuit(n_qubits=n_qubits)
    trotter_circ = gen_term_sequence_circuit(
        scaled_hamiltonian, reference_state=base_circ
    )
    trotter_circ.name = "Trotter step"
    trotter_box = CircBox(trotter_circ)
    for _ in range(n_trotter_steps):
        time_evo_circ.add_circbox(trotter_box, range(n_qubits))

    return CircBox(time_evo_circ)

In [5]:
def get_first_order_trotter_step(
    qubit_pauli_operator: QubitPauliOperator, Delta_t: float, qubits: list[Qubit]
):
    # Bug in docstring of gen_term_sequence_circuit -> no minus required!
    scaled_hamiltonian = Delta_t * (2 / np.pi) * qubit_pauli_operator
    n_qubits = len(qubits)

    base_circ = Circuit(n_qubits=n_qubits)
    trotter_circ = gen_term_sequence_circuit(
        scaled_hamiltonian, reference_state=base_circ
    )
    trotter_circ.name = "1st-order Trotter step"

    return CircBox(trotter_circ)


def get_second_order_trotter_step(
    qubit_pauli_operator: QubitPauliOperator, Delta_t: float, qubits: list[Qubit]
):
    circ = Circuit(name="2nd-order Trotter step")
    reg_trotterbox = circ.add_q_register("reg_trotterbox", len(qubits))

    qubit_pauli_strings = list(qubit_pauli_operator._dict.keys())
    coefficients = list(qubit_pauli_operator._dict.values())

    pauli_exp_boxes = []
    for qps, coef in zip(qubit_pauli_strings[:-1], coefficients[:-1]):
        pauli_exp_boxes.append(
            get_pauli_exp_box_from_QubitPauliString(coef / 2, Delta_t, qps, qubits)
        )

    pauli_exp_boxes.append(
        get_pauli_exp_box_from_QubitPauliString(
            coefficients[-1], Delta_t, qubit_pauli_strings[-1], qubits
        )
    )

    for pauli_exp_box in pauli_exp_boxes[:-1]:
        circ.add_pauliexpbox(pauliexpbox=pauli_exp_box, qubits=reg_trotterbox.to_list())
    circ.add_pauliexpbox(
        pauliexpbox=pauli_exp_boxes[-1], qubits=reg_trotterbox.to_list()
    )
    for pauli_exp_box in pauli_exp_boxes[-2::-1]:
        circ.add_pauliexpbox(pauliexpbox=pauli_exp_box, qubits=reg_trotterbox.to_list())

    box = CircBox(circ)

    return box

In [6]:
def get_first_order_trotterization(
    hamiltonian: QubitPauliOperator,
    n_qubits: int,
    t_trotterization: float,
    n_trotter_steps: int,
):
    trotter_step_size = t_trotterization / n_trotter_steps

    qubits = [Qubit(i) for i in range(n_qubits)]

    time_evo_circ = Circuit(n_qubits=n_qubits, name="Time evolution")
    trotter_box = get_first_order_trotter_step(hamiltonian, trotter_step_size, qubits)
    for _ in range(n_trotter_steps):
        time_evo_circ.add_circbox(trotter_box, range(n_qubits))

    return CircBox(time_evo_circ)


def get_second_order_trotterization(
    hamiltonian: QubitPauliOperator,
    n_qubits: int,
    t_trotterization: float,
    n_trotter_steps: int,
):
    trotter_step_size = t_trotterization / n_trotter_steps

    qubits = [Qubit(i) for i in range(n_qubits)]

    time_evo_circ = Circuit(n_qubits=n_qubits, name="Time evolution")
    trotter_box = get_second_order_trotter_step(hamiltonian, trotter_step_size, qubits)
    for _ in range(n_trotter_steps):
        time_evo_circ.add_circbox(trotter_box, range(n_qubits))

    return CircBox(time_evo_circ)


In [9]:
def trotter_error_vs_steps(
    n_qubits: int = 4,
    J_ZZ: float = 1.2,
    t: float = 1.0,
    trotter_steps_list=None,
    order="first"
):
    """
    Build Trotter circuits of given 'order' ("first" or "second") for 
    the TFIM Hamiltonian. Compare final states to the exact evolution 
    under H for time t. Return errors in a list.
    """
    if trotter_steps_list is None:
        # e.g. 1, 21, 41, 61, 81, 101, 121, 141, 161, 181, 201
        trotter_steps_list = list(range(1, 202, 20))

    # 1) Build the Hamiltonian
    from mytrotter import get_transverse_field_ising_hamiltonian
    H_tfim = get_transverse_field_ising_hamiltonian(n_qubits, J_ZZ)

    # 2) Exact final state from classical expm
    #    We start from |psi0> = |0...0>, which in vector form is e_0 = [1, 0, 0, ...].
    from mytrotter import hamiltonian_time_evolution_numpy
    U_exact = hamiltonian_time_evolution_numpy(H_tfim, t, n_qubits)

    dim = 2**n_qubits
    psi0 = np.zeros(dim, dtype=complex)
    psi0[0] = 1.0  # state |0...0>
    psi_exact = U_exact @ psi0

    # 3) Prepare a backend to get final states from Trotter circuits
    backend = AerStateBackend()

    errors = []
    # 4) Loop over different p = number of Trotter steps
    for p in trotter_steps_list:
        # Build the Trotter circuit box
        if order == "first":
            from mytrotter import get_first_order_trotterization
            evo_box = get_first_order_trotterization(H_tfim, n_qubits, t, p)
            circuit_name = f"TFIM_1st_order_p{p}"
        elif order == "second":
            from mytrotter import get_second_order_trotterization
            evo_box = get_second_order_trotterization(H_tfim, n_qubits, t, p)
            circuit_name = f"TFIM_2nd_order_p{p}"
        else:
            raise ValueError("order must be 'first' or 'second'")

        # Build a main circuit, apply the Trotter circuit box to qubits
        circ = Circuit(n_qubits, name=circuit_name)
        circ.add_circbox(evo_box, list(range(n_qubits)))
        # No measurement gates needed, we just want the final state
        # Decompose to standard gates if needed:
        circ.DecomposeBoxes()

        # 5) Run on state backend to get final Trotter state
        handle = backend.run_circuit(circ)
        result = backend.get_result(handle)
        psi_trotter = result.get_state()

        # 6) Compute the error
        #    e.g. L2 norm of difference: || psi_exact - psi_trotter ||
        err = np.linalg.norm(psi_exact - psi_trotter)
        errors.append(err)

    return trotter_steps_list, errors

In [8]:
def main():
    n_qubits = 4
    J_ZZ = 1.2
    t = 1.0
    p_values = list(range(1, 202, 20))  # 1,21,41,...

    # Get errors for 1st-order Trotter
    p_1st, errors_1st = trotter_error_vs_steps(
        n_qubits=n_qubits, J_ZZ=J_ZZ, t=t, trotter_steps_list=p_values, order="first"
    )

    # Get errors for 2nd-order Trotter
    p_2nd, errors_2nd = trotter_error_vs_steps(
        n_qubits=n_qubits, J_ZZ=J_ZZ, t=t, trotter_steps_list=p_values, order="second"
    )

    # Plot on a double-log scale
    plt.figure(figsize=(7,5))
    plt.loglog(p_1st, errors_1st, "o-", label="1st-order Trotter")
    plt.loglog(p_2nd, errors_2nd, "s-", label="2nd-order Trotter")
    plt.title(f"Trotter Error vs. Number of Steps (N={n_qubits}, t={t})")
    plt.xlabel("Number of Trotter steps, p (log scale)")
    plt.ylabel("Error (||psi_exact - psi_trotter||) (log scale)")
    plt.legend()
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'mytrotter'